# Boosting Homework 
***
**Name**: $<$Juan Lin$>$ 
***

This assignment is due on Moodle by **5pm on Friday April 13th**. Submit only this Jupyter notebook to Moodle.  Do not compress it using tar, rar, zip, etc. Your solutions to analysis questions should be done in Markdown directly below the associated question.  Remember that you are encouraged to discuss the problems with your instructors and classmates, but **you must write all code and solutions on your own**.  For a refresher on the course **Collaboration Policy** click [here](https://github.com/chrisketelsen/CSCI5622-Machine-Learning/blob/master/resources/syllabus.md#collaboration-policy)



## Overview 
***



In this homework you'll implement the AdaBoost classification framework to do handwritten digit recognition. Your implementation should be based on the description of AdaBoost given in the lecture slides.

<br>

![digits](mnist.png "mnist data")


<br>

Here are the rules: 

- Do **NOT** use sklearn's implementation of Adaboost.  You may however use sklearn's implementation of decisions trees. 
- Do **NOT** load or use any Python packages that are not available in Anaconda 3.6. 
- Some problems with code may be autograded.  If we provide a function or class API **do not** change it.
- Do not change the location of the data or data directory.  Use only relative paths to access the data. 

In [ ]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.base import clone 
import matplotlib.pylab as plt
%matplotlib inline 

### [5 points] Problem 1
***

Since we'll be working with binary classifiers, we'll look at the subset of the MNIST data pertaining to handwritten three's and eights. Note that we'll also be using a lower-res version of the MNIST data used in the KNN homework. The class below will load, parse, and store the subset of the. Load the data and then report: 

- The number of examples in the training set 
- The number of examples in the validation set 
- The number of pixels in each image. 

In [ ]:
class ThreesAndEights:
    """
    Class to store MNIST data
    """

    def __init__(self, location):

        import pickle, gzip

        # Load the dataset
        f = gzip.open(location, 'rb')

        # Split the data set 
        X_train, y_train, X_valid, y_valid = pickle.load(f)

        # Extract only 3's and 8's for training set 
        self.X_train = X_train[np.logical_or( y_train==3, y_train == 8), :]
        self.y_train = y_train[np.logical_or( y_train==3, y_train == 8)]
        self.y_train = np.array([1 if y == 8 else -1 for y in self.y_train])
        
        # Shuffle the training data 
        shuff = np.arange(self.X_train.shape[0])
        np.random.shuffle(shuff)
        self.X_train = self.X_train[shuff,:]
        self.y_train = self.y_train[shuff]

        # Extract only 3's and 8's for validation set 
        self.X_valid = X_valid[np.logical_or( y_valid==3, y_valid == 8), :]
        self.y_valid = y_valid[np.logical_or( y_valid==3, y_valid == 8)]
        self.y_valid = np.array([1 if y == 8 else -1 for y in self.y_valid])
        
        print("the number of examples in the training set is:", len(self.X_train))
        print("the number of examples in the validation set is:", len(self.X_valid))
        print("the number of pixels in each image is:", self.X_train.shape[1])
       
        f.close()

In [ ]:
data = ThreesAndEights("./mnist21x21_3789.pklz")
data

### [20 points] Problem 2: Implementing AdaBoost  
***

We've given you a skeleton of the class `AdaBoost` below which will train a classifier based on boosted shallow decision trees as implemented by sklearn. Take a look at the class skeleton first so that you understand the underlying organization and data structures that we'll be using.  

In [ ]:
class AdaBoost:
    def __init__(self, n_learners=20, base=DecisionTreeClassifier(max_depth=1), random_state=1234):
        """
        Create a new adaboost classifier.
        
        Args:
            N (int, optional): Number of weak learners in classifier.
            base (BaseEstimator, optional): Your general weak learner 
            random_state (int, optional): set random generator.  needed for unit testing. 

        Attributes:
            base (estimator): Your general weak learner 
            n_learners (int): Number of weak learners in classifier.
            alpha (ndarray): Coefficients on weak learners. 
            learners (list): List of weak learner instances. 
        """
        
        np.random.seed(random_state)
        
        self.n_learners = n_learners 
        self.base = base
        self.alpha = np.zeros(self.n_learners)
        self.learners = []
        
    def fit(self, X_train, y_train):
        """
        Train AdaBoost classifier on data. Sets alphas and learners. 
        
        Args:
            X_train (ndarray): [n_samples x n_features] ndarray of training data   
            y_train (ndarray): [n_samples] ndarray of data 
        """
        
        # TODO 

        # Note: You can create and train a new instantiation 
        # of your sklearn decision tree as follows 
        # define w_i equals to 1/m
        w = np.ones(len(y_train)) / len(y_train)
        
        # define err_k equals to 0
        err_k = np.zeros(self.n_learners)
        
        for k in range(self.n_learners):
            
            h = clone(self.base)
            self.learners.append(h)
            
            h.fit(X_train, y_train, sample_weight=w)
            h_predict = h.predict(X_train)
            
        # compute weighted error err_k
            for i in range(len(y_train)):
                if h_predict[i] != y_train[i]:
                    err_k[k] = err_k[k] + float((w[i]) / sum(w))
        # compute alpha_k
            self.alpha[k] = (1/2) * np.log(float(1 - err_k[k]) / err_k[k])
        
        # update w_i
            for i in range(len(y_train)):
                if h_predict[i] == y_train[i]:
                    w[i] = w[i] * np.exp(-self.alpha[k])
                else:
                    w[i] = w[i] * np.exp(self.alpha[k])
        
        # make sure w = 1 using normalization
            w = w / sum(w)
        
                 
            
    def predict(self, X):
        """
        Adaboost prediction for new data X.
        
        Args:
            X (ndarray): [n_samples x n_features] ndarray of data 
            
        Returns: 
            yhat (ndarray): [n_samples] ndarray of predicted labels {-1,1}
        """

        # TODO 

        prediction = np.zeros(X.shape[0])
        for k in range(self.n_learners):
            prediction += self.alpha[k] * self.learners[k].predict(X)
        prediction = np.sign(prediction)
            
        return prediction
    
    def score(self, X, y):
        """
        Computes prediction accuracy of classifier.  
        
        Args:
            X (ndarray): [n_samples x n_features] ndarray of data 
            y (ndarray): [n_samples] ndarray of true labels  
            
        Returns: 
            Prediction accuracy (between 0.0 and 1.0).
        """

        
        # TODO 
        score = 0
        prediction_all = self.predict(X)
        for item in range(len(y)):
            if prediction_all[item] == y[item]:
                score += 1
        score = float(score/len(y))

        return score
        
    
    def staged_score(self, X, y):
        """
        Computes the ensemble score after each iteration of boosting 
        for monitoring purposes, such as to determine the score on a 
        test set after each boost.
        
        Args:
            X (ndarray): [n_samples x n_features] ndarray of data 
            y (ndarray): [n_samples] ndarray of true labels  
            
        Returns: 
            scores (ndarary): [n_learners] ndarray of scores 
        """

        # TODO 
        st_score = np.zeros(self.n_learners)
        prediction_k = np.zeros(len(y))
        
        for k in range(self.n_learners):
            prediction_k += self.alpha[k] * self.learners[k].predict(X)
            prediction_k_sign = np.sign(prediction_k)
            for k1 in range(len(y)):
                if prediction_k_sign[k1] == y[k1]:
                    st_score[k] += 1
            st_score[k] = float(st_score[k]/len(y))


        return  st_score
    
    def staged_margin(self, x, y):
        """
        Computes the staged margin after each iteration of boosting 
        for a single training example x and true label y
        
        Args:
            x (ndarray): [n_features] ndarray of data 
            y (integer): an integer {-1,1} representing the true label of x 
            
        Returns: 
            margins (ndarary): [n_learners] ndarray of margins 
        """
        
        # TODO 
        
        margins = []
        margin_diff = 0
        alpha_sum = 0

        for k in range(0, len(self.learners)):
            y_predict = self.learners[k].predict([x])
            alpha_sum += self.alpha[k]
            alpha_hat = self.alpha[k] / alpha_sum
            if y_predict == y:
                margin_diff += alpha_hat
            else:
                margin_diff -= alpha_hat
            margins.append(margin_diff)
       
        return margins
        
        
        

Recall that the model we attempt to learn in AdaBoost is given by 

$$
H({\bf x}) = \textrm{sign}\left[\displaystyle\sum_{k=1}^K\alpha_k h_k({\bf x}) \right]
$$

where $h_k({\bf x})$ is the $k^\textrm{th}$ weak learner and $\alpha_k$ is it's associated ensemble coefficient.  

**Part A**: Implement the `fit` method to learn the sequence of weak learners $\left\{h_k({\bf x})\right\}_{k=1}^K$ and corresponding coefficients $\left\{ \alpha_k\right\}_{k=1}^K$. Note that you may use sklearn's implementation of DecisionTreeClassifier as your weak learner which allows you to pass as an optional parameter the weights associated with each training example.  An example of instantiating and training a single learner is given in the comments of the `fit` method.  

When you think you're done, run the following unit tests which corresponds to the AdaBoost example given in the lecture slides. 


In [ ]:
%run -i tests.py "part A"

**Part B**: After your `fit` method is working properly, implement the `predict` method to make predictions for unseen examples stored in a data matrix ${\bf X}$.  

**Note**: Remember that AdaBoost assumes that your predictions are of the form $y \in \{-1, 1\}$. 

When you think you're done, run the following unit tests which corresponds to the AdaBoost example given in the lecture slides. 

In [ ]:
%run -i tests.py "part B"

**Part C**: Next, implement the `score` method which takes in a matrix of examples ${\bf X}$ and their associated true labels ${\bf y}$, makes predictions, and returns the classification accuracy.   

When you think you're done, run the following unit tests. 

In [ ]:
%run -i tests.py "part C"

**Part D**: Finally, implement the `staged_score` method to return an array of prediction accuracies after each iteration of the AdaBoost algorithm.  That is, the staged score array ${\bf s}$ is defined such that ${\bf s}_\ell$ is the prediction accuracy using only the first $\ell$ weak learners.  This function is primarily used as a diagnostic tool for analyzing the performance of your classifier during the training process.  

**Note**: This method can be implemented in a very efficient or very **in**efficient matter.  Be sure to think about this a bit before diving in. 


When you think you're done, run the following unit tests. 

In [ ]:
%run -i tests.py "part D"

### [10 points] Problem 3: AdaBoost for Handwritten Digit Recognition 
***

Use your AdaBoost code with Sklearn's DecisionTreeClassifier as the base learner to distinguish $3$'s from $8$'s. 
Run $n=500$ boosting iterations with trees of depths 1, 2, and 3 (go deeper if you like) as the weak learner. For each weak learner, plot the training and validation error per boosting iteration (on the same set of axes). Compare and contrast the different weak learners. Which works the best? Do you see signs of overfitting? Do any of classifiers achieve nearly 100% accuracy on the training data? What happens to the accuracy on the validation data on further iterations?

**Problem 3 Answer**                                                                                                               
As shown in the picture below, tree depth = 1 works best, the overfitting starts much slower than the other two depths, and the error between training data and testing data is not that big compared to the other two depths. It is kinda a good bias and viariance trade-off. For all these three depths 1, 2, 3, I have seen signs of overfitting when they happen at different iteration round. For depth 1, it happens at around iteration round 200, for depth 2, it happens at around 30, and for depth 3, it happens at the very beginning. When depth is equal to 3, the training data can get a nearly 100% accuracy. Given further iterations, the accuracy on the test data increases slowly, tend to be stable. 

In [ ]:
data = ThreesAndEights("./mnist21x21_3789.pklz")


In [ ]:
labels_1 = ['train accuracy with depth 1', 'train accuracy with depth 2', 'train accuracy with depth 3']
labels_2 = ['validation accuracy with depth 1', 'validation accuracy with depth 2', 'validation accuracy with depth 3']

for i in range(3):
    digit_num = AdaBoost(n_learners=500, base=DecisionTreeClassifier(max_depth=i+1))
    digit_num.fit(data.X_train, data.y_train)
    
    training_error = digit_num.staged_score(data.X_train, data.y_train)
    valid_error = digit_num.staged_score(data.X_valid, data.y_valid)
    x = np.linspace(0, 500, len(training_error))
    plt.plot(x, training_error, color="blue", label=labels_1[i])
    plt.plot(x, valid_error, color="red", label=labels_2[i])
    plt.xlabel("iteration")
    plt.ylabel("accuracy")
    plt.legend(loc = 'lower right')
    plt.show()

### [15 points] Problem 4: AdaBoost as a Margin-Maximizing Model 
***

Despite the fact that we're making our model more complex with the addition of each weak learner, AdaBoost does not typically overfit the training data. The reason for this is that the model becomes more _confident_ with each boosting iteration. This _confidence_ can be interpreted mathematically as a margin. Recall that after $K$ iterations the algorithm terminates with the classifier 

$$
H({\bf x}) = \textrm{sign}\left[\displaystyle\sum_{k=1}^K\alpha_k h_k({\bf x}) \right]
$$

Similarly, we can define the intermediate classifier $H_\ell$ by 

$$
H_\ell({\bf x}) = \textrm{sign}\left[\displaystyle\sum_{k=1}^\ell\alpha_k h_k({\bf x}) \right]
$$

where $\ell \leq K$. Note that in either case the model returns predictions of the form $y \in \{-1, 1\}$ which does not give us any indication of the model's confidence in a prediction. Define the normalized coefficients $\hat{\alpha}_k$ as follows: 

$$
\hat{\alpha}_k = \dfrac{\alpha_k}{\sum_{t=1}^K \alpha_k}
$$

Define the margin of a training example ${\bf x}$ after $\ell$ iterations as the sum of the normalized coefficients of weak learners that vote correctly minus the sum of the normalized coefficients of the weak learners that vote incorrectly: 

$$
\textrm{margin}_\ell ({\bf x}) = \sum_{k=1:~h_k({\bf x}) = y}^\ell \hat{\alpha}_k - \sum_{k=1:~h_k({\bf x}) \neq y}^\ell \hat{\alpha}_k 
$$

**Part A**: Briefly explain mathematically how $\textrm{margin}_\ell({\bf x})$ can be interpreted as a margin.  **Hint**: You'll want to think back on what we meant by a _margin_ in our discussion of Support Vector Machines. 

**Problem 4, Part A Answer** : In SVM, the margin was defined as 
<br>
$$
{\bf w}^T{\bf x^+} + b = +1
$$
<br>
<br>
$$
{\bf w}^T{\bf x^-} + b = -1
$$
<br>
<br>
$$
{\bf x^+} = {\bf x^-} + {\lambda}w
$$
<br>
<br>$${\bf M} = {\bf x^-} + {\lambda}w$$<br>
<br>$$M = {\bf |x^+ - x^-|} = |{\lambda}w| = \frac{1}{\|{\bf w}\|}$$<br>
So SVM is to maximize distance from the decision boundary to closet training datapoints and this closet distance is called the margin. Here $\textrm{margin}_\ell({\bf x})$ is to maximize the distance of the sum of the weak learners which vote correctly and the sum of the weak learners which vote incorrectly. Therefore we could interpret it as a margin.

**Part B**: Complete the `staged_margin` method in the `AdaBoost` class above so that it computes the margin for a single training example ${\bf x}$ after each boosting iteration. 

**Part C**: Find four **training** examples from the MNIST that meet the following criteria: 

- one $3$ that AdaBoost can classify easily  
- one $8$ that AdaBoost can classify easily  
- one $3$ that AdaBoost has difficulty with 
- one $8$ that AdaBoost has difficulty with 

Use the `view_digit` function given below to display the four examples that you found. 

**Advice**: Since AdaBoost will likely classify **all** training examples correctly given enough boosting iterations, you might try fitting an AdaBoost classifier with just a handful of boosting iterations and use it to identify examples of each desired type. 

**Problem 4, Part C Answer**:                                                                                                                 
one 3 that AdaBoost can classify easily is data.X_train[1]                                                                        
one 8 that AdaBoost can classify easily is data.X_train[3]                                                                              
one 3 that AdaBoost has difficulty with is data.X_train[105]                                                                          
one 8 that AdaBoost has difficulty with is data.X_train[107]

In [ ]:
def view_digit(example, label=None):
    if label: 
        print("true label: {:d}".format(label))
    plt.imshow(example.reshape(21,21), cmap='gray');
    
#view_digit(data.X_train[3,:], data.y_train[3])
view_digit(data.X_train[9,:], data.y_train[9])

In [ ]:
digit_num = AdaBoost(n_learners=10, base=DecisionTreeClassifier(max_depth=2))
digit_num.fit(data.X_train, data.y_train)
diff = digit_num.predict(data.X_train) - data.y_train
temp = []
for i in range(len(diff)):
    if diff[i] != 0:
        temp.append(i)
print(temp)
view_digit(data.X_train[3,:], data.y_train[3])

**Part D**: Using an AdaBoost classifier with at least $K=200$ depth-1 decision trees as the weak learners, plot the staged margin for each of the four examples that you found in **Part C** on the same set of axes. (Be sure to include a legend so we can tell which staged margin corresponds to which example).  Explain your results in terms of the margin of the classifier on each training examples.  More broadly, how the margin-maximizing property might allow AdaBoost to continue improving generalization even after the error on the training set reaches zero.  


**Problem 4, Part D Answer:**
Without any doubt,the staged margin for the items (3 easily classfied and 8 easily classified) have higher values than that of the items (3 hardly classfied and 8 hardly classify). Also the staged margin tends to change quite a bit in the first 50 iterations with the item of 8 easily classified compared to the item 3 easily classified and 3 hardly classified.                 
When the training error is zero, then all examples are on the "right side" of the linear combination, here we view the hyperplan in a feature space where each base hypothesis is one feature/dimension. As soon as the training error is zero, the examples are on the right side and have positive margin. Therefore it is why AdaBoost will still learn even after the training error is zero.

In [ ]:
digit_num = AdaBoost(n_learners=200, base=DecisionTreeClassifier(max_depth=1))
digit_num.fit(data.X_train, data.y_train)

margin3_3 = digit_num.staged_margin(data.X_train[1,:], data.y_train[1])
margin8_8 = digit_num.staged_margin(data.X_train[3,:], data.y_train[3])
margin3_8 = digit_num.staged_margin(data.X_train[105,:], data.y_train[105])
margin8_3 = digit_num.staged_margin(data.X_train[107,:], data.y_train[107])
x = np.linspace(0, 200, len(margin3_3))
plt.plot(x, margin3_3, color="blue", label="3 easily classified")
plt.plot(x, margin8_8, color="red", label="8 easily classified")
plt.plot(x, margin3_8, color="green", label="3 hardly classified")
plt.plot(x, margin8_3, color="orange", label="8 hardly classified")
plt.xlabel("iteration")
plt.ylabel("staged_margin")
plt.legend(bbox_to_anchor=(1.05, 1), borderaxespad=0.)
plt.show()